<a href="https://colab.research.google.com/github/dyou-wbg/meta_processing/blob/main/Clean_50k_sample__char_based_distance_knn_deduping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
from google.colab import drive
import csv
import os
import re
import pandas as pd
import numpy as np

directory = '/content/drive/MyDrive/finalMetaFile'

Mounted at /content/drive


In [2]:

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)


<ipython-input-2-1390d13c0cdd>:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [ ]:
# can skip if directly grab the 50k sample file
df = pd.read_csv('/content/drive/MyDrive/All_titles.csv')
df.rename(columns={'0': 'title'}, inplace=True)
df=df.drop(columns=['Unnamed: 0'])
df=df.sample(n=50000, random_state=321).dropna()

# deselect non-english titles

In [ ]:

import nltk
nltk.download('punkt')
from nltk import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# can skip if directly grab the 50k sample file
# select when title is at least >=2 character and have 70% of them in english
def remove_non_english(df):
  def is_english(text):
    words = word_tokenize(text)
    english_word_count = sum(len(word) >= 2 for word in words)
    return english_word_count / len(words) >= 0.7

  df_filtered = df[df['title'].apply(is_english)]
  return df_filtered

df_english_titles = remove_non_english(df.copy())

print("Original DataFrame:")
print(len(df))
print("\nDataFrame with English titles only:")
print(len(df_english_titles))

Original DataFrame:
49898

DataFrame with English titles only:
48173


In [ ]:
# can skip if directly grab the 50k sample file
df_english_titles = df_english_titles[~df_english_titles['title'].str.contains('و|д|я', regex=True)]

In [ ]:
len(df_english_titles)

47554

In [ ]:
df_english_titles = pd.read_csv('/content/drive/MyDrive/finalMetaFile/50k_sample.csv')

In [ ]:
df_english_titles=df_english_titles[~df_english_titles['title'].str.lower().isin(["conclusion", "poetry","introduction","front matter","editorial","back matter","correspondence","ist of Abbreviations","Book Reviews","Notifiable Diseases and Mortality Tables"])]
len(df_english_titles)

47526

In [ ]:
df_english_titles.to_csv(os.path.join(directory, '50k_sample.csv'), index=False)

In [ ]:
len(df_english_titles)/50000

0.95052

# from here

In [3]:
df_english_titles = pd.read_csv('/content/drive/MyDrive/finalMetaFile/50k_sample.csv')

In [4]:
df_english_titles['title'] = df_english_titles['title'].str.lower()

In [5]:
len(df_english_titles)

47526

In [6]:
df_english_titles.drop_duplicates(subset='title', keep='first')

In [8]:
df_english_titles.drop_duplicates()

In [10]:
print("===========================")
duplicate_rows = df_english_titles[df_english_titles.duplicated(subset='title')]
print(duplicate_rows)

print("===========================")
len(duplicate_rows)/len(df_english_titles)

                                                                                                                                                       title
3176   book reviews                                                                                                                                         
3631   book reviews                                                                                                                                         
3991   calcutta diary                                                                                                                                       
5076   work and employment                                                                                                                                  
6105   england and wales                                                                                                                                    
6872   proliferation of sub-regional trade agreements in t

0.0031351260362748813

In [11]:
df_english_titles_temp = df_english_titles.drop(duplicate_rows.index)

In [12]:
len(df_english_titles_temp)/len(df_english_titles)

0.9968648739637251

In [13]:
df_english_titles=df_english_titles_temp

# fuzzy matching

In [15]:
from string import printable
from collections import Counter

printable_dict = {c:i for i,c in enumerate(printable)}

def vectorize(title):
    cnt = Counter(title)
    array = [0 for _ in printable]+[0] # Last position is for nonprintable characters
    for c,n in cnt.items():
        if c in printable_dict:
            array[printable_dict[c]]=n
        else:
            array[-1]+=n # Update last position if it's not printable
    return np.array(array)

vects = []
for title in df_english_titles.title:
    vects.append(vectorize(title))

In [ ]:
%%time
threshold = 10
min_length = 20
# Iterate over title pairs
for i, v1 in enumerate(vects):
    if sum(v1[:-1]) > min_length: # Only above a minimum number of printable characters
        for j, v2 in enumerate(vects):
            if sum(v2[:-1]) > min_length:
                if i!=j:
                    l1 = np.linalg.norm((v1[:-1] - v2[:-1]), ord=1)
                    if l1 < threshold:
                        t1 = df_english_titles.iloc[i].title
                        t2 = df_english_titles.iloc[j].title
                        print(t1)
                        print(t2)
                        print('_'*100)


Streaming output truncated to the last 5000 lines.
causes of banking crises revisited
____________________________________________________________________________________________________
is rate classification profitable
international politics of africa
____________________________________________________________________________________________________
inflation more devastating to hill country
the information value of bond ratings
____________________________________________________________________________________________________
inflation more devastating to hill country
the constitutional history of medieval england
____________________________________________________________________________________________________
inflation more devastating to hill country
the etiology of revolutions in latin america
____________________________________________________________________________________________________
multiscale contextual poverty in the netherlands
technological developments in the 